# Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [31]:
import pandas as pd
import plotly.express as px
import numpy as np #for NaN handling
import plotly.graph_objects as go

# Loading and Initial Data Inspection

In [35]:
# Load CSV file
try:
    df = pd.read_csv('/content/drive//MyDrive/Colab Notebooks/WHO-COVID-19-Global-Data.csv')
except FileNotFoundError:
    print("Error: 'your_new_file.csv' not found. Please ensure the CSV file is in the same directory or provide the full path.")
    exit()
df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0


# Data Preprocessing and Animated Visualization on World Map

In [36]:
# Ensure Date_reported is in datetime format
df['Date_reported'] = pd.to_datetime(df['Date_reported'])

# Filter dataset from 2019 to the latest available date
df = df[df['Date_reported'].dt.year >= 2019]

# Fill NaN values in numerical columns
df['New_cases'] = df['New_cases'].fillna(0)
df['Cumulative_cases'] = df['Cumulative_cases'].fillna(0)
df['New_deaths'] = df['New_deaths'].fillna(0)
df['Cumulative_deaths'] = df['Cumulative_deaths'].fillna(0)

# Group by Date_reported and Country, aggregating cases and deaths
animated_df = df.groupby(['Date_reported', 'Country']).agg({
    'Cumulative_cases': 'sum',
    'Cumulative_deaths': 'sum',
}).reset_index()

# Sort data by date to ensure proper animation sequence
animated_df = animated_df.sort_values('Date_reported')

# Plot with animation frame on daily dates
fig = px.scatter_geo(animated_df,
                     locations='Country',
                     locationmode='country names',
                     hover_name='Country',
                     size='Cumulative_cases',
                     color='Cumulative_deaths',
                     animation_frame=animated_df['Date_reported'].dt.strftime('%Y-%m-%d'),
                     color_continuous_scale='Reds',
                     size_max=50,
                     title='Global Spread of COVID (2019–{}) Over Time'.format(animated_df['Date_reported'].dt.year.max()))

fig.update_layout(geo=dict(showframe=False, showcoastlines=True))
fig.show()

# Top 10 Countries by Cumulative Cases and Deaths

In [37]:
# Get the latest cumulative data for each country
latest_data_per_country = df.groupby('Country').agg(
    Cumulative_cases=('Cumulative_cases', 'max'),
    Cumulative_deaths=('Cumulative_deaths', 'max')
).reset_index()

# Sort by Cumulative_cases to find the top countries
top_10_countries = latest_data_per_country.sort_values(by='Cumulative_cases', ascending=False).head(10)

# Create a grouped bar chart for cumulative cases and deaths
fig1 = go.Figure(data=[
    go.Bar(name='Cumulative Cases', x=top_10_countries['Country'], y=top_10_countries['Cumulative_cases'], marker_color='blue'),
    go.Bar(name='Cumulative Deaths', x=top_10_countries['Country'], y=top_10_countries['Cumulative_deaths'], marker_color='red')
])

fig1.update_layout(
    barmode='group',
    title_text='Top 10 Countries by Cumulative Cases and Deaths (as of Latest Data)',
    xaxis_title='Country',
    yaxis_title='Count',
    legend_title='Metric'
)
fig1.show()

# Top 10 Countries with Most Deaths

In [38]:
# Sort by Cumulative_deaths to find the top 10 countries with most deaths
top_10_deaths_countries = latest_data_per_country.sort_values(by='Cumulative_deaths', ascending=False).head(10)

# Create a bar chart for cumulative deaths
fig = px.bar(top_10_deaths_countries,
             x='Country',
             y='Cumulative_deaths',
             color='Cumulative_deaths',
             color_continuous_scale=px.colors.sequential.Reds,
             title='Top 10 Countries by Cumulative Deaths (as of Latest Data)',
             labels={'Cumulative_deaths': 'Total Deaths'},
             text='Cumulative_deaths'
            )

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis_title='Country', yaxis_title='Total Deaths', uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()